In [4]:
import pandas as pd

# Función para limpiar nombres de columnas
def clean_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

try:
    # Cargar los archivos CSV con el delimitador correcto (coma)
    indicators_df = pd.read_csv('SDG-indicators.csv', delimiter=',')
    targets_df = pd.read_csv('SDG-targets.csv', delimiter=',')

    # Limpiar los nombres de las columnas
    indicators_df = clean_column_names(indicators_df)
    targets_df = clean_column_names(targets_df)

    # Mostrar las columnas para depuración
    print("Columnas en SDG-indicators.csv:", indicators_df.columns.tolist())
    print("Columnas en SDG-targets.csv:", targets_df.columns.tolist())

    # Renombrar las columnas relevantes
    indicators_df = indicators_df.rename(columns={
        'goal': 'goal_id',
        'target': 'target_id',
        'description': 'indicator_description'
    })

    targets_df = targets_df.rename(columns={
        'goal': 'goal_id',
        'target': 'target_id',
        'description': 'target_description'
    })

    # Verificar si las columnas se han renombrado correctamente
    print("\nColumnas renombradas en SDG-indicators.csv:", indicators_df.columns.tolist())
    print("Columnas renombradas en SDG-targets.csv:", targets_df.columns.tolist())

    # Agrupar los indicadores por goal_id y target_id y concatenar sus descripciones
    indicators_grouped = indicators_df.groupby(['goal_id', 'target_id'])['indicator_description'].apply(lambda x: ' | '.join(x)).reset_index()

    # Unir los datos de targets con los indicadores agrupados
    merged_df = pd.merge(targets_df, indicators_grouped, on=['goal_id', 'target_id'], how='left')

    # Crear la columna 'texto' concatenando la descripción del target con los indicadores
    merged_df['texto'] = merged_df.apply(
        lambda row: f"{row['target_description']} | {row['indicator_description']}" 
        if pd.notna(row['indicator_description']) else row['target_description'],
        axis=1
    )

    # Seleccionar las columnas deseadas
    final_df = merged_df[['goal_id', 'target_id', 'texto']]

    # Guardar el resultado en un nuevo archivo CSV
    final_df.to_csv('SDG_final.csv', index=False, encoding='utf-8')

    print("\nEl archivo 'SDG_final.csv' ha sido creado exitosamente.")

except FileNotFoundError as e:
    print(f"Error: {e}. Asegúrate de que los archivos CSV están en el directorio correcto.")
except pd.errors.ParserError as e:
    print(f"Error al parsear el archivo CSV: {e}. Verifica el delimitador y el formato del archivo.")
except KeyError as e:
    print(f"Error de clave: {e}. Verifica los nombres de las columnas en tus archivos CSV.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")


Columnas en SDG-indicators.csv: ['goal', 'target', 'indicator', 'description', 'tier_proposed', 'tier_revised', 'tier_proposed_coded', 'tier_revised_coded']
Columnas en SDG-targets.csv: ['goal', 'target', 'description']

Columnas renombradas en SDG-indicators.csv: ['goal_id', 'target_id', 'indicator', 'indicator_description', 'tier_proposed', 'tier_revised', 'tier_proposed_coded', 'tier_revised_coded']
Columnas renombradas en SDG-targets.csv: ['goal_id', 'target_id', 'target_description']

El archivo 'SDG_final.csv' ha sido creado exitosamente.


In [6]:
# Nombre del archivo CSV generado previamente
archivo_csv = 'SDG_final.csv'

# Definir el goal_id fijo
goal_id_fijo = '1'  # Cambia este valor según el GOAL que desees (por ejemplo, '2', '3', etc.)

try:
    # Cargar el archivo CSV
    df = pd.read_csv(archivo_csv, delimiter=',', encoding='utf-8')
    print("Archivo CSV cargado exitosamente.\n")
    
    # Verificar que las columnas necesarias existen en el DataFrame
    columnas_necesarias = {'goal_id', 'target_id', 'texto'}
    if not columnas_necesarias.issubset(df.columns):
        missing = columnas_necesarias - set(df.columns)
        raise KeyError(f"Faltan las siguientes columnas en el archivo CSV: {missing}")
    
    # Filtrar las filas que corresponden al goal_id especificado
    df_goal = df[df['goal_id'].astype(str) == str(goal_id_fijo)]
    
    # Verificar si se encontraron filas para el goal_id especificado
    if df_goal.empty:
        print(f"No se encontraron registros para el goal_id '{goal_id_fijo}'.")
    else:
        # Extraer la columna 'texto' y almacenarla en una lista llamada 'targets'
        targets = df_goal['texto'].tolist()
        
        # Mostrar la cantidad de textos extraídos
        print(f"Se han extraído {len(targets)} textos para el goal_id {goal_id_fijo}.\n")
        
        # Mostrar los textos extraídos
        print("Lista 'targets':")
        for idx, texto in enumerate(targets, start=1):
            print(f"{idx}. {texto}\n")
        
        # (Opcional) Guardar la lista 'targets' en un archivo de texto
        # Descomenta las siguientes líneas si deseas guardar los textos en un archivo
        """
        nombre_archivo_salida = f'targets_goal_{goal_id_fijo}.txt'
        with open(nombre_archivo_salida, 'w', encoding='utf-8') as f:
            for texto in targets:
                f.write(texto + '\n')
        print(f"Lista 'targets' guardada en el archivo '{nombre_archivo_salida}'.")
        """

except FileNotFoundError:
    print(f"Error: El archivo '{archivo_csv}' no se encontró en el directorio actual.")
except pd.errors.EmptyDataError:
    print(f"Error: El archivo '{archivo_csv}' está vacío.")
except pd.errors.ParserError:
    print(f"Error: El archivo '{archivo_csv}' no pudo ser parseado. Verifica el delimitador y el formato del archivo.")
except KeyError as e:
    print(f"Error de clave: {e}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Archivo CSV cargado exitosamente.

Se han extraído 7 textos para el goal_id 1.

Lista 'targets':
1. By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day | Proportion of population below the international poverty line, by sex, age, employment status and geographical location (urban/rural)

2. By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions  | Proportion of population living below the national poverty line, by sex and age | Proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions

3. Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable  | Proportion of population covered by social protection floors/systems, by sex, distinguish

In [3]:
import pandas as pd

# Nombre del archivo CSV
archivo_csv = 'Compiled-Keywords-for-SDG.csv'

# Definir el parámetro fijo X (por ejemplo, 'SDG 1')
X = 'SDG 1'  # Cambia este valor según el GOAL que desees (e.g., 'SDG 2', 'SDG3', etc.)

try:
    # Cargar el archivo CSV
    df = pd.read_csv(archivo_csv, delimiter=',', encoding='utf-8')
    print(f"Archivo '{archivo_csv}' cargado exitosamente.\n")
    
    # Verificar que las columnas X y 'Misc' existen en el DataFrame
    columnas_necesarias = [X, 'Misc']
    columnas_presentes = df.columns.tolist()
    
    for columna in columnas_necesarias:
        if columna not in columnas_presentes:
            raise KeyError(f"La columna '{columna}' no existe en el archivo CSV. Columnas disponibles: {columnas_presentes}")
    
    # Extraer las columnas X y 'Misc', eliminando valores nulos y espacios en blanco
    elementos_X = df[X].dropna().astype(str).str.strip()
    elementos_Misc = df['Misc'].dropna().astype(str).str.strip()
    
    # Combinar ambas series
    combinados = pd.concat([elementos_X, elementos_Misc])
    
    # Eliminar entradas vacías después de stripping y convertir a lista de elementos únicos
    combinados = combinados[combinados != '']
    palabras_clave = combinados.unique().tolist()
    
    # Eliminar posibles duplicados adicionales y valores vacíos
    palabras_clave = [elemento for elemento in palabras_clave if elemento]
    
    # Mostrar la lista 'palabras_clave'
    print(f"Lista 'palabras_clave' para la columna '{X}' y 'Misc':\n")
    for idx, elemento in enumerate(palabras_clave, start=1):
        print(f"{idx}. {elemento}")
    
    # (Opcional) Guardar la lista 'palabras_clave' en un archivo de texto
    # Descomenta las siguientes líneas si deseas guardar los resultados
    """
    nombre_archivo_salida = f'palabras_clave_{X.replace(" ", "_")}_y_Misc.txt'
    with open(nombre_archivo_salida, 'w', encoding='utf-8') as f:
        for elemento in palabras_clave:
            f.write(elemento + '\n')
    print(f"\nLa lista 'palabras_clave' ha sido guardada en el archivo '{nombre_archivo_salida}'.")
    """

except FileNotFoundError:
    print(f"Error: El archivo '{archivo_csv}' no se encontró en el directorio actual.")
except pd.errors.EmptyDataError:
    print(f"Error: El archivo '{archivo_csv}' está vacío.")
except pd.errors.ParserError:
    print(f"Error: El archivo '{archivo_csv}' no pudo ser parseado. Verifica el delimitador y el formato del archivo.")
except KeyError as e:
    print(f"Error de clave: {e}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")


Archivo 'Compiled-Keywords-for-SDG.csv' cargado exitosamente.

Lista 'palabras_clave' para la columna 'SDG 1' y 'Misc':

1. Africa
2. Basic services
3. Class
4. Developing countries
5. Disadvantaged
6. Economic resources
7. End poverty
8. Environment
9. Equality
10. Extreme poverty
11. Financial inclusion
12. Income
13. Income equality
14. Microfinance
15. Non-discrimination
16. Poor
17. Poor and vulnerable
18. Poverty
19. Poverty eradication
20. Poverty line
21. Quality of Life
22. Resources
23. Social protection systems / social protection
24. Sustainable
25. Third World
26. Vulnerable
27. Wealth distribution
28. Accountability
29. Alternative energy
30. Biodiversity
31. Capacity building
32. Caring for country
33. CO2 emissions
34. Disability
35. Eco tourism
36. Ecology
37. Energy efficiency
38. Environmental
39. Environmental degradation
40. Environmental policy
41. Environmental sustainability
42. Equal rights to economic resources
43. Ethical
44. Food-energy-water nexus / Water-e

In [2]:
targets

['Africa',
 'Basic services',
 'Class',
 'Developing countries',
 'Disadvantaged',
 'Economic resources',
 'End poverty',
 'Environment',
 'Equality',
 'Extreme poverty',
 'Financial inclusion',
 'Income',
 'Income equality',
 'Microfinance',
 'Non-discrimination',
 'Poor',
 'Poor and vulnerable',
 'Poverty',
 'Poverty eradication',
 'Poverty line',
 'Quality of Life',
 'Resources',
 'Social protection systems / social protection',
 'Sustainable',
 'Third World',
 'Vulnerable',
 'Wealth distribution',
 'Accountability',
 'Alternative energy',
 'Biodiversity',
 'Capacity building',
 'Caring for country',
 'CO2 emissions',
 'Disability',
 'Eco tourism',
 'Ecology',
 'Energy efficiency',
 'Environmental',
 'Environmental degradation',
 'Environmental policy',
 'Environmental sustainability',
 'Equal rights to economic resources',
 'Ethical',
 'Food-energy-water nexus / Water-energy-food nexus',
 'Forced displacement',
 'Good governance',
 'Governance',
 'Governance and policy',
 'Governan